In [ ]:
import os
import torch
from Models.MultiViewViT import MultiViewViT
from load_data import IMG_Folder
import torch.nn as nn

In [ ]:
def weights_init(w):
    classname = w.__class__.__name__
    if classname.find('Conv') != -1:
        if hasattr(w, 'weight'):
            # nn.init.kaiming_normal_(w.weight, mode='fan_out', nonlinearity='relu')
            nn.init.kaiming_normal_(w.weight, mode='fan_in', nonlinearity='leaky_relu')
        if hasattr(w, 'bias') and w.bias is not None:
                nn.init.constant_(w.bias, 0)
    if classname.find('Linear') != -1:
        if hasattr(w, 'weight'):
            torch.nn.init.xavier_normal_(w.weight)
        if hasattr(w, 'bias') and w.bias is not None:
            nn.init.constant_(w.bias, 0)
    if classname.find('BatchNorm') != -1:
        if hasattr(w, 'weight') and w.weight is not None:
            nn.init.constant_(w.weight, 1)
        if hasattr(w, 'bias') and w.bias is not None:
            nn.init.constant_(w.bias, 0)

In [ ]:
# Load model
model = MultiViewViT(
    image_sizes=[(91, 109), (91, 91), (109, 91)],
    patch_sizes=[(7, 7), (7, 7), (7, 7)],
    num_channals=[91, 109, 91],
    vit_args={
        'emb_dim': 768, 'mlp_dim': 3072, 'num_heads': 12,
        'num_layers': 12, 'num_classes': 1,
        'dropout_rate': 0.1, 'attn_dropout_rate': 0.0
    },
    mlp_dims=[3, 128, 256, 512, 1024, 512, 256, 128, 1]
)
model.apply(weights_init)
model = model.to("cpu")

# Load checkpoint
CheckpointPath = r'C:\Users\Rishabh\training_output_metricsMulti_VIT_best_model.pth.tar'
checkpoint = torch.load(CheckpointPath, map_location="cpu")
state_dict = checkpoint["state_dict"]
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)

In [ ]:
import pandas as pd
CheckpointPath = r'C:\Users\Rishabh\trainingMulti_VIT_best_model.pth.tar'
CSVPath = r'C:\Users\Rishabh\Documents\TransBTS\IXI.xlsx'
DataFolder = r'C:\Users\Rishabh\Documents\TrimeseData'
device = "cpu"
Files = os.listdir(DataFolder)
ixi_ids = [int(f[3:6]) for f in Files]

In [ ]:
df = pd.read_excel(CSVPath)

In [ ]:
attentionsv1 = []       # [B,H,T,T] per layer (forward)
attention_gradsv1 = []  # [B,H,T,T] per layer (backward)
attentionsv2 = []       # [B,H,T,T] per layer (forward)
attention_gradsv2 = []  # [B,H,T,T] per layer (backward)
attentionsv3 = []       # [B,H,T,T] per layer (forward)
attention_gradsv3 = []  # [B,H,T,T] per layer (backward)

def hook_attn_probs_v1(module, inp, out):
    # out might be probs or (ctx, probs) depending on your code
    probs = out[1] if isinstance(out, (tuple, list)) else out
    attentionsv1.append(probs)                    # DO NOT detach if you want grads
    probs.register_hook(lambda g: attention_gradsv1.append(g))  # tensor-level grad hook

def hook_attn_probs_v2(module, inp, out):
    # out might be probs or (ctx, probs) depending on your code
    probs = out[1] if isinstance(out, (tuple, list)) else out
    attentionsv2.append(probs)                    # DO NOT detach if you want grads
    probs.register_hook(lambda g: attention_gradsv2.append(g))  # tensor-level grad hook


def hook_attn_probs_v3(module, inp, out):
    # out might be probs or (ctx, probs) depending on your code
    probs = out[1] if isinstance(out, (tuple, list)) else out
    attentionsv3.append(probs)                    # DO NOT detach if you want grads
    probs.register_hook(lambda g: attention_gradsv3.append(g))  # tensor-level grad hook

# Register on each block's attention where probs exist
for blk in model.vit_1.transformer.encoder_layers:
    blk.attn.register_forward_hook(hook_attn_probs_v1)

# Register on each block's attention where probs exist
for blk in model.vit_2.transformer.encoder_layers:
    blk.attn.register_forward_hook(hook_attn_probs_v2)

# Register on each block's attention where probs exist
for blk in model.vit_3.transformer.encoder_layers:
    blk.attn.register_forward_hook(hook_attn_probs_v3)

In [ ]:

def white0(image, threshold=0):
    """
    Standardize voxels with value > threshold

    Args:
        image: Input image
        threshold: Threshold value

    Returns:
        Standardized image
    """
    image = image.astype(np.float32)
    mask = (image > threshold).astype(int)

    # Vectorized implementation to avoid unnecessary memory allocation
    image_h = image * mask

    # Calculate mean and std only for relevant voxels
    non_zero_voxels = np.sum(mask)
    if non_zero_voxels > 0:
        mean = np.sum(image_h) / non_zero_voxels

        # More memory efficient way to calculate std
        std_sum = np.sum((image_h - mean * mask) ** 2)
        std = np.sqrt(std_sum / non_zero_voxels)

        if std > 0:
            normalized = mask * (image - mean) / std
            # Use in-place operations to reduce memory usage
            image = normalized + image * (1 - mask)
            return image

    # Default case
    return np.zeros_like(image, dtype=np.float32)


In [ ]:
len(Files)

In [ ]:
import nibabel as nib
import numpy as np
model.eval()
idx = 16
for idx in range(100,  len(Files)):
    filename = Files[idx]
    file_path = os.path.join(DataFolder, filename)
    img = nib.load(file_path)
    x_np = img.get_fdata(caching='unchanged').astype(np.float32)       # avoid float64 bloat
    # x_tn = torch.from_numpy(x_np).unsqueeze(0).to(device).float()
    # print(type(inputvolume), inputvolume.device, inputvolume.shape)
    
    print(x_np.shape)
    
    _id = int(filename[3:6])
    # AGE = df[df['IXI_ID']==_id]['AGE'].values[0]
    
    # inputvolume = x_tn.to(device).type(torch.FloatTensor)
    inputvolume = white0(x_np)
    inputvolume = torch.from_numpy(inputvolume).unsqueeze(0).to(device).float()
    inputvolume = inputvolume.to(device).type(torch.FloatTensor)
    model.zero_grad(set_to_none=True)

    output, (attn1, attn2, attn3) = model(inputvolume, return_attention_weights=True)
    output.backward()
    Predicted_Age = output.item()
    del x_np, inputvolume, output, attn1, attn2, attn3
    torch.cuda.empty_cache()

In [ ]:
Attnv1 = torch.stack(attentionsv1)
AttnGrv1 = torch.stack(attention_gradsv1)
Attnv1 = torch.mean(Attnv1, dim=0)
AttnGrv1 = torch.mean(AttnGrv1, dim=0)
Attnv1 = torch.mean(Attnv1, dim=0)
AttnGrv1 = torch.mean(AttnGrv1, dim=0)

Attnv2 = torch.stack(attentionsv2)
AttnGrv2 = torch.stack(attention_gradsv2)
Attnv2 = torch.mean(Attnv2, dim=0)
AttnGrv2 = torch.mean(AttnGrv2, dim=0)
Attnv2 = torch.mean(Attnv2, dim=0)
AttnGrv2 = torch.mean(AttnGrv2, dim=0)


Attnv3 = torch.stack(attentionsv3)
AttnGrv3 = torch.stack(attention_gradsv3)
Attnv3 = torch.mean(Attnv3, dim=0)
AttnGrv3 = torch.mean(AttnGrv3, dim=0)
Attnv3 = torch.mean(Attnv3, dim=0)
AttnGrv3 = torch.mean(AttnGrv3, dim=0)

Attnv1.shape, AttnGrv1.shape,Attnv2.shape, AttnGrv2.shape, Attnv3.shape, AttnGrv3.shape

In [ ]:
import numpy as np
import torch
def grad_rollout(attentions, gradients, discard_ratio):
    result = torch.eye(attentions[0].size(-1))
    with torch.no_grad():
        for attention, grad in zip(attentions, gradients):                
            weights = grad
            attention_heads_fused = attention*weights
            print(attention_heads_fused.shape)
            attention_heads_fused = attention_heads_fused.mean(axis=1)
            attention_heads_fused[attention_heads_fused < 0] = 0

            # Drop the lowest attentions, but
            # don't drop the class token
            flat = attention_heads_fused.view(attention_heads_fused.size(0), -1)
            _, indices = flat.topk(int(flat.size(-1)*discard_ratio), -1, False)
            #indices = indices[indices != 0]
            flat[0, indices] = 0

            I = torch.eye(attention_heads_fused.size(-1))
            a = (attention_heads_fused + 1.0*I)/2
            a = a / a.sum(dim=-1)
            result = torch.matmul(a, result)
    
    # Look at the total attention between the class token,
    # and the image patches
    return result

In [ ]:
resultv1 = grad_rollout(Attnv1, AttnGrv1, 0.9)
resultv2 = grad_rollout(Attnv2, AttnGrv2, 0.9)
resultv3 = grad_rollout(Attnv3, AttnGrv3, 0.9)

In [ ]:
import torch.nn.functional as F
import cv2

vec1 = resultv1[0, 1:]  # (T_no_cls,)
vec2 = resultv2[0, 1:]  # (T_no_cls,)
vec3 = resultv3[0, 1:]  # (T_no_cls,)
print(len(vec1),len(vec2),len(vec3))
mask1 = vec1.reshape(15, 13).detach().numpy()
mask1 = cv2.resize(mask1 / mask1.max(), (109, 91))

mask2 = vec2.reshape(13, 13).detach().numpy()
mask2 = cv2.resize(mask2 / mask2.max(), (91, 91))

mask3 = vec3.reshape(13, 15).detach().numpy()
mask3 = cv2.resize(mask3 / mask3.max(), (91, 109))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(mask1, cmap='hot')  # mask is your 2D NumPy array
plt.colorbar(label='Attention intensity')
plt.title('Attention Map')
plt.axis('off')
plt.show()

In [ ]:
idx = 23
filename = Files[idx]
file_path = os.path.join(DataFolder, filename)
img = nib.load(file_path)
x_np = img.get_fdata(caching='unchanged').astype(np.float32)       # avoid float64 bloat

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example: pick a middle slice along z-axis
slice_idx = x_np.shape[2] // 2  

img_slice = x_np[:, :, slice_idx]
print(img_slice.shape)
mask_slice = mask1

plt.figure(figsize=(10,5))

# Show base image
plt.imshow(img_slice, cmap='gray')

# Overlay mask with transparency
plt.imshow(mask_slice, cmap='jet', alpha=0.5)  # alpha controls overlay strength
plt.colorbar(label="Mask intensity")
plt.axis("off")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example: pick a middle slice along z-axis
slice_idx = x_np.shape[0] // 2  

img_slice = x_np[slice_idx, :, :]
mask_slice = mask3

plt.figure(figsize=(10,5))

# Show base image
plt.imshow(img_slice, cmap='gray')

# Overlay mask with transparency
plt.imshow(mask_slice, cmap='jet', alpha=0.5)  # alpha controls overlay strength
plt.colorbar(label="Mask intensity")
plt.axis("off")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example: pick a middle slice along z-axis
slice_idx = x_np.shape[0] // 2  

img_slice = x_np[:, slice_idx, :]
mask_slice = mask2

plt.figure(figsize=(10,5))

# Show base image
plt.imshow(img_slice, cmap='gray')

# Overlay mask with transparency
plt.imshow(mask_slice, cmap='jet', alpha=0.5)  # alpha controls overlay strength
plt.colorbar(label="Mask intensity")
plt.axis("off")
plt.show()